In [31]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='./../.env')
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'),temperature=0.7)
dspy.configure(lm=lm)

In [2]:
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

Lee, a 21-year-old footballer, made seven appearances for the Hammers, scoring once in a Europa League match. He had loan spells at Blackpool and Colchester United, scoring twice but could not help Colchester avoid relegation. His contract details with the Tykes remain undisclosed.


## signature

In [26]:
class PersonalizedSummary(dspy.Signature):
    document: str = dspy.InputField(desc="The document to summarize")
    length: int = dspy.InputField(desc="The maximum length of the summary")
    content_preference: str = dspy.InputField(desc="The topic that you want to focus on in the summary")
    style: str = dspy.InputField(default="formal", desc="The style of the summary, either 'formal' or 'casual'")
    wiki_knowledge: str = dspy.InputField(default=False, desc="May contain relevant facts, include the information in the summary")
    summary: str = dspy.OutputField(desc="The summary of the document")
    
# summarize = dspy.ChainOfThought(PersonalizedSummary)
# response = summarize(document=document,length=50,content_preference="football", style="casual", wiki_knowledge="")
# print(response.summary)

## LLM evaluation

In [46]:
class Assess(dspy.Signature):
    summary_text = dspy.InputField(desc="The summary text, evaluated based on whether the length, content, and style personalization preferences are met")
    length: int = dspy.InputField(desc="The maximum length of the summary")
    content_preference: str = dspy.InputField(desc="The topic that you want to focus on in the summary")
    style: str = dspy.InputField(default="formal", desc="The style of the summary, either 'formal' or 'casual'")
    score: int = dspy.OutputField(desc="The score of the summary, between 0 and 100")

class Compare(dspy.Signature):
    summary_text = dspy.InputField(desc="The summary text")
    ground_truth_summary = dspy.InputField(desc="The ground truth summary text")
    score: int = dspy.OutputField(desc="The score of the summary text, between 0 and 100")
    

def assess(summary_text, length, content_preference, style):
    score=dspy.Predict(Assess)(summary_text=summary_text,length=length,content_preference=content_preference,style=style).score
    return score

def metric(summary_text1, summary_text2):
    score=dspy.Predict(Compare)(summary_text=summary_text1,ground_truth_summary=summary_text2).score
    return score

In [10]:
text="Lee, a 21-year-old footballer, made seven appearances for the Hammers, scoring once in a Europa League match. He had loan spells at Blackpool and Colchester United, scoring twice but could not help Colchester avoid relegation. His contract details with the Tykes remain undisclosed."
assess(summary_text=response.summary,length=50,content_preference="football", style="casual")

70

## Background knowledge

### add retriever

In [27]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(rm=colbertv2_wiki17_abstracts)

class KeyWords(dspy.Signature):
    document: str = dspy.InputField(desc="The document to extract keywords from")
    keywords: list = dspy.OutputField(desc="The top 10 keywords extracted from the document")

class SummaryPipeline(dspy.Module):
    def __init__(self, wiki_knowledge=False):
        self.wiki_knowledge = wiki_knowledge
        if wiki_knowledge:
            self.retriever = dspy.Retrieve(k=3)
            self.keywords = dspy.ChainOfThought(KeyWords)
        self.summarizer = dspy.ChainOfThought(PersonalizedSummary)
    
    def forward(self, document, content_preference="", length=50, style="formal"):
        wiki_knowledge=""
        if self.wiki_knowledge:
            key_words = self.keywords(document=document).keywords
            # Retrieve relevant documents
            wiki_knowledge = self.retriever(query=key_words).passages
        # Summarize the relevant documents
        summary = self.summarizer(document=document, content_preference=content_preference, length=length, style=style, wiki_knowledge=wiki_knowledge).summary
        return summary

## Examples

In [28]:
summary_pipeline = SummaryPipeline(wiki_knowledge=True)
response = summary_pipeline(document=document)

In [35]:
response

'The 21-year-old footballer made seven appearances for the Hammers, scoring once in a Europa League match. He had loan spells at Blackpool and Colchester United, netting twice for the latter. His contract details with the Tykes remain undisclosed.'

In [34]:
metric(summary_text="",length=50,content_preference="football", style="formal")

60

## Optimization

In [47]:
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""
length=50
content_preference="football"
style="formal"
summary='The 21-year-old footballer made seven appearances for the Hammers, scoring once in a Europa League match. He had loan spells at Blackpool and Colchester United, netting twice for the latter. His contract details with the Tykes remain undisclosed.'

trainset=[dspy.Example(document=document,length=length,content_preference=content_preference,style=style,summary=summary).with_inputs("document", "content_preference", "length", "style")]
evaluate=dspy.Evaluate(devset=trainset,metric=metric)
summary_pipeline=SummaryPipeline(wiki_knowledge=False)
evaluate(summary_pipeline)

2024/12/01 22:15:45 INFO dspy.evaluate.evaluate: Average Metric: 85 / 1 (8500.0%)


8500.0

In [48]:
from dspy.teleprompt import BootstrapFewShot
teleprompt = BootstrapFewShot(metric=metric)
compiled_predictor=teleprompt.compile(summary_pipeline,trainset=trainset)

100%|██████████| 1/1 [00:00<00:00, 288.03it/s]

Bootstrapped 0 full traces after 0 examples for up to 1 rounds, amounting to 1 attempts.


In [49]:
evaluate(compiled_predictor)

2024/12/01 22:17:38 INFO dspy.evaluate.evaluate: Average Metric: 85 / 1 (8500.0%)


8500.0